# LOAN PREDICTION DATASET

Among all industries, the insurance domain has one of the largest uses of analytics & data science methods. This dataset provides you a taste of working on data sets from insurance companies – what challenges are faced there, what strategies are used, which variables influence the outcome, etc. This is a classification problem. The data has 615 rows and 13 columns.

### Required libraries

In [65]:
#Import required libraries for data analysis and wrangling

import pandas as pd
import numpy as np
import random as rnd 

# Impord required libraries for data visualization

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Import required ML libraries

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### read the datasets

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9bbfa4ab4e0548258fb2cc9e70eef4a9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='golVVT44CVe8IcH4xyb6bpf4Pjr4jb7o5z_cgqpKXsbL',
    ibm_auth_endpoint="https://iam.au-syd.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_9bbfa4ab4e0548258fb2cc9e70eef4a9.get_object(Bucket='loanpredictionmodel-donotdelete-pr-wwcz4ixhsdedie',Key='train_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
train_data = pd.read_csv(body)
train_data.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:

body = client_9bbfa4ab4e0548258fb2cc9e70eef4a9.get_object(Bucket='loanpredictionmodel-donotdelete-pr-wwcz4ixhsdedie',Key='test_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
test_data = pd.read_csv(body)
test_data.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [4]:
test_data.shape

(367, 12)

In [5]:
train_data.shape

(614, 13)

#### Combine whole data for future processing

In [6]:
combine = [train_data,test_data]

In [7]:
train_data.columns.values

array(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area', 'Loan_Status'], dtype=object)

In [8]:
test_data.columns.values

array(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area'], dtype=object)

In [9]:
train_data.info()

print("*"*100)

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB
****************************************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               356 non-null object
Married 

### FIND FEATURE COLUMNS

#### First convert categorial feature to numeric and drop unwanted columns

In [10]:
freq_occur_gender = train_data.Gender.mode()[0]
freq_occur_gender

'Male'

##### Replcae Nan value to frequently occoured value of Gender

In [11]:
for dataset in combine:
    dataset['Gender'] = dataset['Gender'].fillna(freq_occur_gender)

In [12]:
for dataset in combine:
    dataset['Gender']= dataset['Gender'].map({'Female':1,'Male':0}).astype(int)

In [13]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,0,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,0,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [14]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,0,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,0,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,0,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,0,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


##### Replcae Nan value to frequently occoured value of Marrital Status

In [15]:
freq_occur_marrital_status = train_data.Married.mode()[0]
freq_occur_marrital_status

'Yes'

In [16]:
for dataset in combine:
    dataset['Married'] = dataset['Married'].fillna(freq_occur_marrital_status)
    
# Replcae categorical value to numeric value

for dataset in combine:
    dataset['Married']=dataset['Married'].map({'Yes':1,'No':0}).astype(int)

In [17]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,1,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,0,1,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,1,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,0,0,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [18]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0,1,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,0,1,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,0,1,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,0,1,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,0,0,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


##### Replcae Nan value to frequently occoured value of Education Status

In [19]:
freq_occur_education_status = train_data.Education.mode()[0]
freq_occur_education_status

'Graduate'

In [20]:
for dataset in combine:
    dataset['Education'] = dataset['Education'].fillna(freq_occur_education_status)
    
# Replcae categorical value to numeric value

for dataset in combine:
    dataset['Education']=dataset['Education'].map({'Graduate':1,'Not Graduate':0}).astype(int)

In [21]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,1,1,1,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,0,1,0,1,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,1,0,0,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,0,0,0,1,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [22]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0,1,0,1,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,0,1,1,1,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,0,1,2,1,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,0,1,2,1,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,0,0,0,0,No,3276,0,78.0,360.0,1.0,Urban


##### Replcae Nan value to frequently occoured value of Education Status

In [23]:
freq_occur_Employment_status = train_data.Self_Employed.mode()[0]
freq_occur_Employment_status

'No'

In [24]:
for dataset in combine:
    dataset['Self_Employed'] = dataset['Self_Employed'].fillna(freq_occur_Employment_status)
    
# Replcae categorical value to numeric value

for dataset in combine:
    dataset['Self_Employed']=dataset['Self_Employed'].map({'Yes':1,'No':0}).astype(int)

In [25]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,1,1,1,0,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,0,1,0,1,1,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,1,0,0,0,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,0,0,0,1,0,6000,0.0,141.0,360.0,1.0,Urban,Y


In [26]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0,1,0,1,0,5720,0,110.0,360.0,1.0,Urban
1,LP001022,0,1,1,1,0,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,0,1,2,1,0,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,0,1,2,1,0,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,0,0,0,0,0,3276,0,78.0,360.0,1.0,Urban


In [27]:
ApplicantIncome_mean = train_data['ApplicantIncome'].mean()
ApplicantIncome_mean

5403.459283387622

In [28]:
for dataset in combine:
    dataset['ApplicantIncome']=dataset['ApplicantIncome'].fillna(ApplicantIncome_mean)

In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           599 non-null object
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(5), object(4)
memory usage: 62.4+ KB


##### Replcae Nan value to frequently occoured value of Dependency Status

In [30]:
freq_occur_dependency_status = train_data['Dependents'].mode()[0]
freq_occur_dependency_status

'0'

In [31]:
train_data.Dependents.unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [32]:
for dataset in combine:
    dataset['Dependents'] = dataset['Dependents'].fillna(freq_occur_dependency_status)


In [33]:
train_data.Dependents.unique()

array(['0', '1', '2', '3+'], dtype=object)

In [34]:
for dataset in combine:
    dataset['Dependents'] = dataset['Dependents'].map({'0':0,'1':1,'2':2,'3+':3}).astype(int)

In [35]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,1,1,1,0,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,0,1,0,1,1,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,1,0,0,0,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,0,0,0,1,0,6000,0.0,141.0,360.0,1.0,Urban,Y


In [36]:
train_data.Dependents.unique()

array([0, 1, 2, 3])

In [37]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(6), object(3)
memory usage: 62.4+ KB


##### Replcae Nan value to mean value of CoapplicantIncome

In [38]:
CoapplicantIncome_mean = train_data.CoapplicantIncome.mean()
CoapplicantIncome_mean

1621.245798027101

In [39]:
for dataset in combine:
    dataset['CoapplicantIncome']=dataset['CoapplicantIncome'].fillna(CoapplicantIncome_mean)

In [40]:
for dataset in combine:
    dataset['CoapplicantIncome'] = dataset['CoapplicantIncome'].astype(int)

In [41]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0,NaN,360.0,1.0,Urban,Y
1,LP001003,0,1,1,1,0,4583,1508,128.0,360.0,1.0,Rural,N
2,LP001005,0,1,0,1,1,3000,0,66.0,360.0,1.0,Urban,Y
3,LP001006,0,1,0,0,0,2583,2358,120.0,360.0,1.0,Urban,Y
4,LP001008,0,0,0,1,0,6000,0,141.0,360.0,1.0,Urban,Y


In [42]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null int64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(3), int64(7), object(3)
memory usage: 62.4+ KB


##### Replcae Nan value to mean value of LoanAmount

In [43]:
LoanAmount_mean = train_data.LoanAmount.mean()
LoanAmount_mean

146.41216216216216

In [44]:
for dataset in combine:
    dataset['LoanAmount'] = dataset['LoanAmount'].fillna(LoanAmount_mean)

In [45]:
for dataset in combine:
    dataset['LoanAmount'] = dataset['LoanAmount'].astype(int)

In [46]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0,146,360.0,1.0,Urban,Y
1,LP001003,0,1,1,1,0,4583,1508,128,360.0,1.0,Rural,N
2,LP001005,0,1,0,1,1,3000,0,66,360.0,1.0,Urban,Y
3,LP001006,0,1,0,0,0,2583,2358,120,360.0,1.0,Urban,Y
4,LP001008,0,0,0,1,0,6000,0,141,360.0,1.0,Urban,Y


##### Replcae categorial fetaure into numerical value which name is :"Loan_amount_term"

In [47]:
train_data.Loan_Amount_Term.unique()

array([360., 120., 240.,  nan, 180.,  60., 300., 480.,  36.,  84.,  12.])

In [48]:
freq_occur_loan_amount_term = train_data.Loan_Amount_Term.mode()[0]
freq_occur_loan_amount_term

360.0

In [49]:
for dataset in combine:
    dataset['Loan_Amount_Term']=dataset['Loan_Amount_Term'].fillna(freq_occur_loan_amount_term)

In [50]:
for dataset in combine:
    dataset['Loan_Amount_Term'] = dataset['Loan_Amount_Term'].astype(int)

In [51]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0,146,360,1.0,Urban,Y
1,LP001003,0,1,1,1,0,4583,1508,128,360,1.0,Rural,N
2,LP001005,0,1,0,1,1,3000,0,66,360,1.0,Urban,Y
3,LP001006,0,1,0,0,0,2583,2358,120,360,1.0,Urban,Y
4,LP001008,0,0,0,1,0,6000,0,141,360,1.0,Urban,Y


##### Replcae categorial fetaure into numerical value which name is :"credit_history"

In [52]:
train_data.Credit_History.unique()

array([ 1.,  0., nan])

In [53]:
freq_occur_credit_history = train_data.Credit_History.mode()[0]
freq_occur_credit_history

1.0

In [54]:
for dataset in combine:
    dataset['Credit_History'] = dataset['Credit_History'].fillna(freq_occur_credit_history)

In [55]:
for dataset in combine:
    dataset['Credit_History'] = dataset['Credit_History'].astype(int)

In [56]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null int64
LoanAmount           614 non-null int64
Loan_Amount_Term     614 non-null int64
Credit_History       614 non-null int64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: int64(10), object(3)
memory usage: 62.4+ KB


##### Replcae categorial fetaure into numerical value which name is :"Property_area"

In [57]:
train_data.Property_Area.unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [58]:
freq_occoured_property_area = train_data.Property_Area.mode()[0]
freq_occoured_property_area

'Semiurban'

In [59]:
for dataset in combine:
    dataset['Property_Area'] = dataset['Property_Area'].fillna(freq_occoured_property_area)

In [60]:
for dataset in combine:
    dataset['Property_Area'] =  dataset['Property_Area'].map({'Urban':0,'Rural':1,'Semiurban':3}).astype(int)

In [61]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0,146,360,1,0,Y
1,LP001003,0,1,1,1,0,4583,1508,128,360,1,1,N
2,LP001005,0,1,0,1,1,3000,0,66,360,1,0,Y
3,LP001006,0,1,0,0,0,2583,2358,120,360,1,0,Y
4,LP001008,0,0,0,1,0,6000,0,141,360,1,0,Y


In [62]:
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,0,1,0,1,0,5720,0,110,360,1,0
1,LP001022,0,1,1,1,0,3076,1500,126,360,1,0
2,LP001031,0,1,2,1,0,5000,1800,208,360,1,0
3,LP001035,0,1,2,1,0,2340,2546,100,360,1,0
4,LP001051,0,0,0,0,0,3276,0,78,360,1,0


In [63]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null int64
LoanAmount           614 non-null int64
Loan_Amount_Term     614 non-null int64
Credit_History       614 non-null int64
Property_Area        614 non-null int64
Loan_Status          614 non-null object
dtypes: int64(11), object(2)
memory usage: 62.4+ KB


In [64]:
train_data.Loan_Status.unique()

array(['Y', 'N'], dtype=object)

#### Encode the label term

In [66]:
encode = LabelEncoder()

In [67]:
train_data.Loan_Status = encode.fit_transform(train_data.Loan_Status)

In [68]:
train_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0,0,0,1,0,5849,0,146,360,1,0,1
1,LP001003,0,1,1,1,0,4583,1508,128,360,1,1,0
2,LP001005,0,1,0,1,1,3000,0,66,360,1,0,1
3,LP001006,0,1,0,0,0,2583,2358,120,360,1,0,1
4,LP001008,0,0,0,1,0,6000,0,141,360,1,0,1


In [69]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null int64
LoanAmount           614 non-null int64
Loan_Amount_Term     614 non-null int64
Credit_History       614 non-null int64
Property_Area        614 non-null int64
Loan_Status          614 non-null int64
dtypes: int64(12), object(1)
memory usage: 62.4+ KB


## Visualized data using plotting